<center> 
    <h1> Toronto Crime </h1>
<h3> Sprint 3</h3>
<h3> Final Model</h3>
    </center>

##### Author: Arsalan Arif Radhu
##### Date: 24 November 2023

### Table of Contents:
- [Introduction](#intro) <br />
- [KNN](#knn)<br />
- [XGBoost](#xgb)<br />
- [Random Forest](#rf)<br />

### Introduction <a id="intro" /> 

After optimizing the models, I have decided to go ahead with the XGBoost model as it had the best Accuracy and decent distribution of the classification metrics.

In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np

In [3]:
# Importing the data
df = pd.read_csv('../Data/FixedFinalDataAdvAn.csv', index_col=0)
df.reset_index(inplace=True)

In [4]:
df.head(10)

,index,OBJECTID,CRIME_TYPE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,OCC_YEAR,OCC_MONTH,OCC_DAY,LONG_WGS84,...,D55,NSA,Apartment,Commercial,Educational,House,NotApplicable,Other,Outside,Transit
0,0,1,0,2014,1,1,1998,6,1,-79.425896,...,0,0,1,0,0,0,0,0,0,0
1,1,2,0,2014,1,1,2014,1,1,-79.376497,...,0,0,1,0,0,0,0,0,0,0
2,2,3,0,2014,1,1,2014,1,1,-79.344839,...,1,0,0,0,0,0,0,0,1,0
3,3,4,0,2014,1,1,2014,1,1,-79.391841,...,0,0,0,1,0,0,0,0,0,0
4,4,5,0,2014,1,1,2014,1,1,-85.488744,...,0,1,0,0,0,0,0,0,1,0
5,5,6,0,2014,1,1,2014,1,1,-79.416594,...,0,0,0,1,0,0,0,0,0,0
6,6,7,0,2014,1,1,2014,1,1,-79.391279,...,0,0,0,1,0,0,0,0,0,0
7,7,8,0,2014,1,1,2014,1,1,-79.345795,...,0,0,0,1,0,0,0,0,0,0
8,8,9,0,2014,1,1,2014,1,1,-79.132919,...,0,0,0,0,0,0,0,0,0,1
9,9,10,0,2014,1,1,2014,1,1,-79.455586,...,0,0,1,0,0,0,0,0,0,0


As discussed in the previous notebook, I will be dropping all the indices.

In [5]:
df.drop('index', axis=1, inplace= True)
df.drop('OBJECTID', axis=1, inplace= True)

In [6]:
shuffled_df = df.sample(frac=1)

In [7]:
shuffled_df['CRIME_TYPE'].value_counts()

CRIME_TYPE
5    160284
0    158497
1     62423
3     26171
4      5707
2      1245
Name: count, dtype: int64

Category 0: Assault <br />
Category 1: Break and Enter <br />
Category 2: Homicide <br />
Category 3: Robbery <br />
Category 4: Shooting <br />
Category 5: Theft <br />

In [8]:
shuffled_df.head(10)

,CRIME_TYPE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,OCC_YEAR,OCC_MONTH,OCC_DAY,LONG_WGS84,LAT_WGS84,DEATH,...,D55,NSA,Apartment,Commercial,Educational,House,NotApplicable,Other,Outside,Transit
102304,0,2019,11,23,2019,11,23,-79.367534,43.659574,0,...,0,0,0,0,0,0,0,0,1,0
1553,0,2014,2,11,2013,10,11,-79.425556,43.651450,0,...,0,0,0,1,0,0,0,0,0,0
412872,1,2022,10,10,2022,10,10,-79.310751,43.680787,0,...,1,0,0,1,0,0,0,0,0,0
238448,5,2014,11,4,2014,11,3,-79.526889,43.647879,0,...,0,0,0,0,0,0,0,0,1,0
7264,0,2014,6,29,2014,6,29,-79.400096,43.645835,0,...,0,0,0,1,0,0,0,0,0,0
356163,1,2014,8,10,2014,8,10,-79.397156,43.700847,0,...,0,0,0,1,0,0,0,0,0,0
144634,0,2022,5,2,2022,5,2,-79.252028,43.727266,0,...,0,0,1,0,0,0,0,0,0,0
215076,5,2018,7,23,2018,7,21,-79.389841,43.670556,0,...,0,0,0,0,0,0,0,0,1,0
7055,0,2014,6,25,2014,6,25,-79.289846,43.776942,0,...,0,0,0,0,0,1,0,0,0,0
301363,5,2021,11,14,2021,10,1,-79.409378,43.760325,0,...,0,0,0,0,0,0,0,0,1,0


After many considerations I will be dropping the `Homicide` and `Shootings` categories as they are very rare cases and can be almost be treated as outliers and may be affecting the accuracy and scores of the model.

Using the best parameters we found in the `Optimization` notebook, I will re-run the model after dropping the two categories.

In [9]:
shuffled_df = shuffled_df[~shuffled_df['CRIME_TYPE'].isin([2, 4])]

In [10]:
shuffled_df['CRIME_TYPE'].value_counts()

CRIME_TYPE
5    160284
0    158497
1     62423
3     26171
Name: count, dtype: int64

In [13]:
shuffled_df['CRIME_TYPE'].replace({3: 2, 5: 3}, inplace=True)

After changing: 
Category 0: Assault <br />
Category 1: Break and Enter <br />
Category 2: Robbery <br />
Category 3: Theft <br />


In [15]:
# Extract features and labels
X_boost = shuffled_df.drop('CRIME_TYPE', axis=1)  # Replace 'label_column' with the actual label column name
y_boost = shuffled_df['CRIME_TYPE']

X_train, X_test, y_train, y_test = train_test_split(X_boost, y_boost, test_size=0.3, random_state=12)

# Set the specified parameters for multi-class classification
num_classes = len(np.unique(y_train))
xgb_params = {
    'learning_rate': 0.1,
    'max_depth': 10,
    'min_child_weight': 8,
    'n_estimators': 625,
    'objective': 'multi:softprob',
    'num_class': num_classes,
}

# Initialize the XGBoost model
xgb_model = XGBClassifier(**xgb_params)  # You can adjust the number of estimators

# Train the model
xgb_model.fit(
    X_train, y_train,
    eval_metric='merror', 
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=10
)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

c:\Users\arifh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\arifh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-merror:0.43936
[1]	validation_0-merror:0.43780
[2]	validation_0-merror:0.43629
[3]	validation_0-merror:0.43612
[4]	validation_0-merror:0.43606
[5]	validation_0-merror:0.43565
[6]	validation_0-merror:0.43477
[7]	validation_0-merror:0.43358
[8]	validation_0-merror:0.43300
[9]	validation_0-merror:0.43250
[10]	validation_0-merror:0.43206
[11]	validation_0-merror:0.43175
[12]	validation_0-merror:0.43103
[13]	validation_0-merror:0.43080
[14]	validation_0-merror:0.43018
[15]	validation_0-merror:0.43000
[16]	validation_0-merror:0.42916
[17]	validation_0-merror:0.42850
[18]	validation_0-merror:0.42770
[19]	validation_0-merror:0.42706
[20]	validation_0-merror:0.42641
[21]	validation_0-merror:0.42597
[22]	validation_0-merror:0.42530
[23]	validation_0-merror:0.42432
[24]	validation_0-merror:0.42358
[25]	validation_0-merror:0.42252
[26]	validation_0-merror:0.42157
[27]	validation_0-merror:0.42050
[28]	validation_0-merror:0.41995
[29]	validation_0-merror:0.41924
[30]	validation_0-me

In [16]:
# Display the results
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Accuracy: 0.6413638483631038
Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.75      0.68     47706
           1       0.51      0.34      0.40     18704
           2       0.48      0.06      0.11      7894
           3       0.69      0.75      0.72     47909

    accuracy                           0.64    122213
   macro avg       0.58      0.47      0.48    122213
weighted avg       0.62      0.64      0.62    122213

